In [ ]:
## IMPORTS

from satpy import Scene, find_files_and_readers
from pyresample import geometry, create_area_def
from satpy.writers import get_enhanced_image
from datetime import datetime
import math
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from satpy import MultiScene
from satpy.multiscene import timeseries
%matplotlib inline
from matplotlib_scalebar.scalebar import ScaleBar
import cmocean
import cmocean.cm as cmo
import netCDF4
import pandas as pd
from copy import deepcopy

In [ ]:
## Definitions of Useful Functions

# convert celcius to fahrenheit
def fahrenheit(x):
    fahrenheit = (x*9/5) + 32
    return fahrenheit

# convert fahrenheit to celcius
def celcius(x):
    celcius = (x - 32) * 5/9
    return celcius

# invert sst values 
def sstinvert(x,type = 'num'):
    if type == 'list':
        pass  #Future implemented feature, for inverting everything in the list  
    elif type == 'num':
        x = -1 * (x/255) # Divide by 225 in order to put on a 0-1 scale, and then multiply by negative one, to start inverting
        x += 1 # Invert the number, i.e. 1 now equals 0, 0 now equals 1. Allows us to highlight low sst values
    return x

In [ ]:
# Define projection and mapping stuff - same for both SST and CDOM
extent=[-94, 27.5, -88, 30.5]
res = '10m'
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -91., 'lat_0': 29.5, 'lat_1': 29.5, 'lat_2': 29.5},
                          width=1500, height=750,
                          area_extent=extent, units='degrees')

In [ ]:
# Load modis images to calculate CDOM
base = "/home/luka/tinker/NASA/NasaInternship/Data_Downloads/24-10-31:24-11-07/*.hdf"
dayfiles = glob(base)
modis = Scene(dayfiles, reader='modis_l1b')
modis.load(['8','4','13lo', 'true_color'])

# Resample MODIS images
modis = modis.resample(my_area)

In [ ]:
# Calculate CDOM

Rss412 = (modis['8'])
Rss555 = (modis['4'])
Rss667 = (modis['13lo'])

B0 = 0.2487; B1 = 14.028; B2 = 4.085
aCDOM412_a = (np.log((Rss412/Rss555 - B0)/ B2))/(-B1)
aCDOM412_a = aCDOM412_a.compute()


# Create the combo array to store sed CDOM in
combo = deepcopy(modis['8'].drop_attrs())

In [ ]:
# Load and resample sst geotiff
tiff = glob('/home/luka/tinker/NASA/NasaInternship/Data_Downloads/24-10-31:24-11-07/*.TIFF')
sst = Scene(tiff, reader='generic_image')
sst.load(['image'])
sst = sst.resample(my_area)

In [ ]:
a = 0
combo.load() # load a sacrifical dataset into memory, to overwrite with the Sed. CDOM index values
for x in range(750): # loop through each of the rows in the datasets
    for y in range(1500): # loop through each of the columns in the dataset
        sstVal = sstinvert(sst['image'][0][x][y]) # Pull the sst value for this specific coordinate, then invert it and store
        cdomVal = float(aCDOM412_a[x][y]) # pull the cdom value for this specififc coordinate
        if math.isnan(cdomVal): # if there is no CDOM value, set to 0
            cdomVal = 0
        combo[x][y] = (cdomVal*sstVal) # multiply the cdom value to the inverted sst value
        # Status stuff below - unimportant
        a+=1
        if a == 100:
            if len((str(x))) == 1:
                px = "00" + str(x)
            elif len((str(x))) == 2:
                px = "0" + str(x)
            else:
                px = str(x)
            if len(str(y)) == 1:
                py = "000" + str(y)
            elif len(str(y)) == 2:
                py = "00" + str(y)
            elif len(str(y)) == 3:
                py = "0" + str(y)
            else:
                py = str(y)
            prn = "(" + px + "," + py + ")"
            print(prn, end='\r')
            a = 0

In [ ]:
manualminmax = True
if manualminmax:
    ## For Black Background
    #cmin = .08
    #cmax = .24
    
    ## For Full Colorscale
    cmin = 0
    cmax = .24
else:
    cmin = float(combo.max())
    cmax = float(combo.min())

fig =  plt.figure(dpi=4000)

ax = plt.axes(projection=crs)
ax.coastlines(res)
plt.pcolormesh(lons, lats, combo, transform=ccrs.PlateCarree(),
              vmin=cmin, vmax=cmax, cmap='inferno')
ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', facecolor='grey',
                                            scale=res))

plt.colorbar(shrink = .6, label = "Sed. CDOM Index")
plt.title('Oct 31-Nov 07 - Sed. CDOM (Full Range)') 
fig.savefig('/home/luka/tinker/NASA/NasaInternship/outputs/Oct 31-Nov 07 SST & CDOM (Full Range) .png')

In [ ]:
cmin = .08
cmax = .24

fig =  plt.figure(dpi=4000)

ax = plt.axes(projection=crs)
ax.coastlines(res)
plt.pcolormesh(lons, lats, combo, transform=ccrs.PlateCarree(),
              vmin=cmin, vmax=cmax, cmap='inferno')
ax.add_feature(cfeature.NaturalEarthFeature(category='physical', 
                                            name='land', facecolor='grey',
                                            scale=res))

plt.colorbar(shrink = .6, label = "Sed. CDOM Index")
plt.title('Oct 31-Nov 07 - Sed. CDOM (High Min)') 
fig.savefig('/home/luka/tinker/NASA/NasaInternship/outputs/Oct 31-Nov 07 SST & CDOM (High Min) .png')